In [3]:
import os
import pandas as pd

os.chdir('F:\\Edwisor\\edwisor exercise\\project\\python')
os.getcwd()
data_bike=pd.read_csv('day.csv')
data_bike.dtypes
data_bike=pd.DataFrame(data_bike)
data_bike.shape


(731, 16)

In [4]:
####### EDA #############
data_bike['dteday']=pd.to_datetime(data_bike.dteday)
data_bike['season']=data_bike.season.astype('category')
data_bike['holiday']=data_bike.holiday.astype('category')
data_bike['yr']=data_bike.yr.astype('category')
data_bike['mnth']=data_bike.mnth.astype('category')
data_bike['workingday']=data_bike.workingday.astype('category')
data_bike['weathersit']=data_bike.weathersit.astype('category')
data_bike['mnth']=data_bike.mnth.astype('category')
data_bike['weekday']=data_bike.weekday.astype('category')

In [ ]:
########## missing value analysis #########
#data_bike.dtypes
data_bike.isnull().sum()

In [ ]:
###Outlier ananlysis
import matplotlib.pyplot as pt
import seaborn as sb
figure,axis=pt.subplots(figsize=(8,8))
sb.boxplot(data=data_bike[['temp','hum','windspeed']])
axis.set_title('outliers from temp,hum,wind')
pt.show()
sb.boxplot(data=data_bike['cnt'])
axis.set_title('count outliers')
pt.show()

In [ ]:
##remove outliers
import numpy as np
df=pd.DataFrame(data_bike,columns=['hum','windspeed'])
c=['hum','windspeed']
for i in c:
    q75,q25=np.percentile(df.loc[:,i],[75,25])
    iqr=q75-q25
    min=q25-(iqr*1.5)
    max=q75+(iqr*1.5)
    df.loc[df.loc[:,i]<min,:i]=np.nan
    df.loc[df.loc[:,i]>max,:i]=np.nan

df.isnull().sum()
df['windspeed']=df['windspeed'].fillna(df['windspeed'].mean())
df['hum']=df['hum'].fillna(df['hum'].mean())
df.isnull().sum()

In [ ]:
data_bike['windspeed']=df['windspeed']
data_bike['hum']=df['hum']
figure,axis=pt.subplots(figsize=(8,8))
sb.boxplot(data=data_bike[['temp','hum','windspeed']])
axis.set_title('outliers from temp,hum,wind')
pt.show()
sb.boxplot(data=data_bike['cnt'])
axis.set_title('count outliers')
pt.show()


In [ ]:
correMtr=data_bike[["temp","atemp","hum","windspeed","casual","registered","cnt"]].corr()
mask=np.array(correMtr)
mask[np.tril_indices_from(mask)]=False
sb.heatmap(correMtr,mask=mask,annot=True)
pt.show()

In [ ]:
###feature selection
data_bike.dtypes
bike=pd.DataFrame(data_bike,columns=['season','yr','mnth','weekday','workingday','weathersit','temp','hum','windspeed','cnt'])
bike.shape

In [ ]:
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
train, test = train_test_split(bike, test_size=0.2)
train.shape
test.shape

In [ ]:
#import statsmodels.api as sm
from sklearn import preprocessing,metrics,linear_model
from sklearn.model_selection import cross_val_score,cross_val_predict,train_test_split
#bike.shape
#mlr = sm.OLS(train.iloc[:,8], train.iloc[:,0:8]).fit()
lr_model=linear_model.LinearRegression()
lr_model.fit(train.iloc[:,0:8],train.iloc[:,-1])

In [ ]:
lr=lr_model.score(train.iloc[:,0:8],train.iloc[:,-1])
print('Accuracy of the model :',lr)
print('Model coefficients :',lr_model.coef_)
print('Model intercept value :',lr_model.intercept_)

In [ ]:
predict_lr=lr_model.predict(test.iloc[:,0:8])
def MAPE(y_true, y_pred): 
    mape = np.mean(np.abs((y_true - y_pred) / y_true))*100
    return mape
#Calculate MAPE
MAPE(test.iloc[:,9], predict_lr)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor(n_estimators=200)

In [ ]:
rf.fit(train.iloc[:,0:8],train.iloc[:,-1])
rf.score(train.iloc[:,0:8],train.iloc[:,-1])


In [ ]:
predict_rf=rf.predict(test.iloc[:,0:8])
MAPE(test.iloc[:,9], predict_rf)

In [ ]:
op_actual=pd.DataFrame(test.iloc[:,9],columns=['cnt'])
#op_actual
op_rf=pd.DataFrame(predict_rf,columns=['predicted value by Rf'])
op_rf
#op_lr=pd.DataFrame(predict_lr,columns=['predicted value by Lr'])
op_actual=op_actual.reset_index()
op_actual=op_actual['cnt']
op_actual
opf_rf=pd.merge(op_actual,op_rf,left_index=True,right_index=True)
opf_rf
#op_rf
#opf_lr=pd.merge(op_actual,op_lr,left_index=True,right_index=True)
#print(opf_lr)
#print(opf_rf)
opf_rf.to_csv('output_python.csv',index=None,header=True)